# Artnet Tests

### Packages

In [7]:
import asyncio
import argparse
from PIL import ImageColor
from pyartnet import ArtNetNode
from pyartnet.base import universe as Universe
from pyartnet.base import channel as Channel

### Constants

In [2]:
# Setup Constants
DEFAULT_PORT = 6454
DEFAULT_UNIVERSE_ID = 1
DEFAULT_CHANNEL_START_ID = 1
DEFAULT_CHANNEL_WIDTH = 11
# Control Constants
DIMMER_ID = 0
DIMMER_FINE_ID = 1
STROBE_ID = 2
RED_ID = 3
GREEN_ID = 4
BLUE_ID = 5
WHITE_ID = 6
AMBER_ID = 7
UV_ID = 8
PRESET_ID = 9
SOUND_ID = 10
DEFAULT_LIGHT_VALUE = [255,255]+[0]*(DEFAULT_CHANNEL_WIDTH-2)
RESET_VALUE = [0]*DEFAULT_CHANNEL_WIDTH
# Mappings
FIXTURE_TO_ID_DICT = {'Dimmer':DIMMER_ID,'Dimmer_Fine':DIMMER_FINE_ID,
                    'Strobe':STROBE_ID,'Red':RED_ID,'Green':GREEN_ID,
                    'Blue':BLUE_ID,'White':WHITE_ID,'Amber':AMBER_ID,
                    'UV':UV_ID,'Preset':PRESET_ID,'Sound':SOUND_ID}
ID_TO_FIXTURE_DICT = dict([(i,fixture) for fixture,i in FIXTURE_TO_ID_DICT.items()])

### Code

#### Helpers

In [7]:
def init_channels(ip:str,channel_list:list,port=DEFAULT_PORT,
                  universe_id=DEFAULT_UNIVERSE_ID,
                  channel_start_id=DEFAULT_CHANNEL_START_ID,
                  channel_width=DEFAULT_CHANNEL_WIDTH) -> tuple:
    """
    Create the connection and channels with the given parameters
    and turn all lights off.

    :param ip: String encoding of device IP.
    :param channel_list: Ordered list containing the names of the channel to be created.
    :param port: Destination port for communication. (int, default=DEFAULT_PORT).
    :param universe_id: Id of the universe of the setup. (int, default=DEFAULT_UNIVERSE_ID).
    :param channel_start_id: Channel start id. (int, default=DEFAULT_CHANNEL_START_ID).
    :param channel_width: Channel width. (int, default=DEFAULT_CHANNEL_WIDTH).

    :return: Tuple of dictionnaries containing the channel and their states.
    
    """
    # Init connections
    node, universe = create_connection(ip,port=port,universe_id=universe_id)
    # Init Channels
    channel_dict = dict()
    state_dict = dict()
    for idx, channel_name in enumerate(channel_list):
        create_channel(universe,channel_dict,channel_name,
                       channel_start_id+channel_width*idx,
                       channel_width)
        turn_off_channel(channel_dict,channel_name)
        state_dict[channel_name] = DEFAULT_LIGHT_VALUE.copy()
    return channel_dict, state_dict
    
    

def create_connection(ip:str,port=DEFAULT_PORT,universe_id=DEFAULT_UNIVERSE_ID) -> tuple:
    """
    Create instance of ArtNetNode and assign universe id.

    :param ip: String encoding of device IP.
    :param port: Destination port for communication. (int, default=DEFAULT_PORT).
    :param universe_id: Id of the universe of the setup. (int, default=DEFAULT_UNIVERSE_ID).

    :return: Tuple in the form (node, universe).
    
    """
    node = ArtNetNode(ip,port)
    universe = node.add_universe(universe_id)
    return node, universe

def create_channel(universe:Universe, channel_dict:dict,
                   channel_name:str,channel_start_id=DEFAULT_CHANNEL_START_ID,
                   channel_width=DEFAULT_CHANNEL_WIDTH): 
    """
    Create channel in the given universe of size
    'channel_width' and starting at 'channel_start'.
    Then add it in the dictionnary of all channels for
    the universe.

    :param universe: ArtNet Universe object in which the channel should be created.
    :param channel_dict: Dictionnary with all the channels for the universe.
    :param channel_name: String to be used as key in the channel dictionnary.
    :param channel_start_id: Channel start id. (int, default=DEFAULT_CHANNEL_START_ID).
    :param channel_width: Channel width. (int, default=DEFAULT_CHANNEL_WIDTH).

    :return: ArtNet Channel with the given specifications.
    
    """
    if channel_start_id < 1 or channel_width < 1 :
        raise ValueError('The channel start id and channel width should be greater or equal than 1')
    channel = universe.add_channel(start=channel_start_id, width=channel_width)
    channel_dict[channel_name] = channel

def send_rgb(channel_dict:dict, channel_name:str,
             values:list, state_dict:dict):
    """
    Send RGB data to the channel, while keeping other parameters fixed.

    :param channel_dict: Dictionnary with all the channels for the universe.
    :param channel_name: Name of the channel to which the signal should be transmitted.
    :param values: List of RGB values in format: [red_value, green_value, blue_value].
                   The values of each color should be contained in [0,255].
    :param state_dict: Dictionnary with the DMX states of all channels.
    
    """
    if len(values) != 3:
        raise ValueError('The list of values should be of length 3')
    current_state = state_dict[channel_name]
    new_state = current_state.copy()
    new_state[RED_ID:BLUE_ID+1] = values
    state_dict[channel_name] = new_state
    channel = channel_dict[channel_name]
    channel.set_values(new_state)

def send_amber(channel_dict:dict, channel_name:str,
             value:int, state_dict:dict):
    """
    Send RGB data to the channel, while keeping other parameters fixed.

    :param channel_dict: Dictionnary with all the channels for the universe.
    :param channel_name: Name of the channel to which the signal should be transmitted.
    :param value: The value to which should be set the amber channel.
                   The value should be contained in [0,255].
    :param state_dict: Dictionnary with the DMX states of all channels.
    
    """
    current_state = state_dict[channel_name]
    new_state = current_state.copy()
    new_state[AMBER_ID] = value
    state_dict[channel_name] = new_state
    channel = channel_dict[channel_name]
    channel.set_values(new_state)

def send_uv(channel_dict:dict, channel_name:str,
             value:int, state_dict:dict):
    """
    Send RGB data to the channel, while keeping other parameters fixed.

    :param channel_dict: Dictionnary with all the channels for the universe.
    :param channel_name: Name of the channel to which the signal should be transmitted.
    :param value: The value to which should be set the amber channel.
                   The value should be contained in [0,255].
    :param state_dict: Dictionnary with the DMX states of all channels.
    
    """
    current_state = state_dict[channel_name]
    new_state = current_state.copy()
    new_state[UV_ID] = value
    state_dict[channel_name] = new_state
    channel = channel_dict[channel_name]
    channel.set_values(new_state)

def turn_off_channel(channel_dict:dict, channel_name:str):
    """
    Turn off the lights of the given channel by setting all values to zero.

    :param channel_dict: Dictionnary with all the channels for the universe.
    :param channel_name: Name of the channel to which the signal should be transmitted.
    
    """
    channel_dict[channel_name].set_values(LIGHT_OFF_VALUE)

#### Objects

In [13]:
class Light:
    """
    Light object class. A light is model by the channel to which
    it is linked and a state. The latter is a list containing the 
    values for each of the fixture in the channel. 
    
    """
    
    def __init__(self, name:str, channel:Channel):
        """
        Create a Light instance.

        :param name: String identifier for the light. If the light is used in a Group,
                     please be sure to enter unique identifiers.
        :param channel: ArtNet channel object to which the light is bound. 

        """
        self.name = name
        self.channel = channel
        self.state = DEFAULT_LIGHT_VALUE.copy()

    def reset(self):
        """
        Reset the light to its default state, i.e. zero value for each fixture.
        """
        new_state = DEFAULT_LIGHT_VALUE
        self.channel.set_values(new_state)
        self.state = new_state

    def set_fixture_value(self, fixture_id:int, value:int):
        """
        Set the fixture to the given value.

        :param fixture_id: Id of the fixture to be set.
        :param value: Integer value of the fixture, should be contained in [0,255].
        
        """
        if value < 0 or value > 255:
            raise ValueError(f'The value for {ID_TO_FIXTURE_DICT[fixture_id]} should be contained in [0,255]')
        new_state = self.state.copy()
        new_state[fixture_id] = value
        self.channel.set_values(new_state)
        self.state = new_state

    def turn_off(self):
        """
        Turn off the light by setting dimmer to 0.
        """
        self.set_fixture_value(DIMMER_ID,0)
    
    def set_rgb(self, values:list):
        """
        Set the RGB fixtures to the color code given in values.

        :param values: List containing the values for the red, green and blue fixtures.
        
        """
        for idx, fixture_id in enumerate([RED_ID,GREEN_ID,BLUE_ID]):
            self.set_fixture_value(fixture_id,values[idx])

In [24]:
class Group:
    """
    Group object class. A group is modeled by a list
    of lights and a name. Every action applied to the group
    will be executed on all lights present. Each light in the 
    group should be unique and have a unique name.
    
    """

    def __init__(self, name:str, lights=[]):
        """
        Create a Group instance.

        :param name: String identifier for the group of lights.
        :param lights: List containing the initial lights of the group,
                       empty by default. All lights should be unique.
        
        """
        self.name = name
        self.lights = lights
        self.light_names = list(set([l.name for l in lights]))
        if len(self.light_names) != len(self.lights):
            raise ValueError('Duplicate names in the list of lights provided to the Group constructor')
    
    def add_light(self,light):
        """
        Add a light to the existing pool. Must be a new unique light.

        :param light. Light object to add to the existing pool of lights in the group.
        
        """
        if light.name in self.light_name:
            raise ValueError('Tried to add a light whose name is already present in the group')
        self.lights.append(light)
        
    def reset(self):
        """
        Reset all lights in the group to their default states, i.e. zero value for each fixture.
        """
        for l in self.lights:
            l.reset()

    def set_fixture_value(self, fixture_id:int, value:int):
        """
        Set the given fixture to 'value' for all lights in the group.

        :param fixture_id: Id of the fixture to be set.
        :param value: Integer value of the fixture, should be contained in [0,255].
        
        """
        for l in self.lights:
            l.set_fixture_value(fixture_id,value)

    def turn_off(self):
        """
        Turn off all lights in the group by setting dimmer to 0.
        """
        for l in self.lights:
            l.turn_off()
    
    def set_rgb(self, values:list):
        """
        Set the RGB fixtures to the color code given in values for all lights in the group.

        :param values: List containing the values for the red, green and blue fixtures.
        
        """
        for l in self.lights:
            l.set_rgb(values)

### Artnet

In [4]:
channel_dict, state_dict = init_channels('169.254.79.148',['main','ambiance'])

In [12]:
send_rgb(channel_dict,'main',[40,0,5],state_dict)

In [7]:
send_rgb(channel_dict,'ambiance',[100,0,255],state_dict)

In [8]:
send_amber(channel_dict,'main',0,state_dict)

In [9]:
send_uv(channel_dict,'main',255,state_dict)

In [20]:
send_amber(channel_dict,'ambiance',255,state_dict)

---

In [13]:
turn_off_channel(channel_dict,'main')
turn_off_channel(channel_dict,'ambiance')

### Variables

In [3]:
node = ArtNetNode('169.254.79.148',6454)

In [4]:
universe = node.add_universe(1)

In [5]:
channel = universe.add_channel(start=1, width=11)

In [16]:
channel.set_values([255,0,0,120,40,10,80,250,0,0,0])

<Channel 1/11 8bit>

### Async

In [ ]:
async def main_async():
    # Run this code in your async function
    node = ArtNetNode('169.254.79.148',6454)

    # Create universe 0
    universe = node.add_universe(1)

    # Add a channel to the universe which consists of 3 values
    # Default size of a value is 8Bit (0..255) so this would fill
    # the DMX values 1..3 of the universe
    channel = universe.add_channel(start=1, width=11)

    # Fade channel to 255,0,0 in 5s
    # The fade will automatically run in the background
    channel.add_fade([255,0,0,0,255,0,0,0,0,0,0],1000)

    # this can be used to wait till the fade is complete
    await channel

In [ ]:
import PySimpleGUI as sg

picked_color = '#1f77b4'

layout = [[sg.In("", visible=False, enable_events=True, key='set_line_color'),
           sg.ColorChooserButton("", size=(1, 1), target='set_line_color', button_color=(picked_color, picked_color),
                                 border_width=1, key='set_line_color_chooser')],
          [sg.Submit(key='submit')]]

window = sg.Window('Window Title', layout)

while True:
    event, values = window.read()

    if event is None:
        break

    elif event == 'set_line_color':
        picked_color = values[event]
        window['set_line_color_chooser'].Update(button_color=(picked_color, picked_color))

    elif event == 'submit':
        break

window.close()

In [14]:
final_color

'#35e1f4'

In [13]:
ImageColor.getcolor(final_color, "RGB")

(53, 225, 244)

In [15]:
import matplotlib.colors
matplotlib.colors.to_rgb(final_color)

(0.20784313725490197, 0.8823529411764706, 0.9568627450980393)